In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url_basica="https://www.whocc.no/atc_ddd_index/?code="
fin="&showdescription=no"

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}


In [4]:
#Obtener primer caso
url=url_basica+"A30AB"+fin
page=requests.get(url,headers=headers)

In [5]:
#Obtener el contenido de la pagina
soup=BeautifulSoup(page.content,'html.parser')

In [8]:
# Categorías y códigos
categories = soup.find_all('b')
category_codes = []
category_names = []

for category in categories:
    category_link = category.find('a', href=True)
    if category_link:
        category_code = category_link['href'].split('=')[1].split('&')[0]
        category_name = category_link.text.strip()
        category_codes.append(category_code)
        category_names.append(category_name)


# Extraer la info de la tabla
table = soup.find('table')
try:
    rows = table.find_all('tr')
except:
    rows = []

#Pegar todas las filas en una lista
data=[]
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append(cols)

In [9]:
data

[]

In [10]:
import numpy as np
df = pd.DataFrame(data, columns=['atc5_code', 'drug_name', 'ddd', 'unit', 'administration', 'note'])
#Incorprar la categoria y el nombre. Cada elemento debe ir en una sola columna
#Eliminar primera fila
df = df.iloc[1:]
#Vacíos como NaN
df = df.replace(r'^\s*$', np.nan, regex=True)
#Rellenar vacíoos de atc5_code y drug_name con el valor de la fila anterior
df['atc5_code'] = df['atc5_code'].fillna(method='ffill')
df['drug_name'] = df['drug_name'].fillna(method='ffill')
df

,atc5_code,drug_name,ddd,unit,administration,note


In [11]:
#Llevr a un dataframe
df = pd.DataFrame(data, columns=['atc5_code', 'drug_name', 'ddd', 'unit', 'administration', 'note'])
#Eliminar primera fila
df = df.iloc[1:]
#Vacíos como NaN
df = df.replace(r'^\s*$', np.nan, regex=True)
#Rellenar vacíoos de atc5_code y drug_name con el valor de la fila anterior
df['atc5_code'] = df['atc5_code'].fillna(method='ffill')
df['drug_name'] = df['drug_name'].fillna(method='ffill')
#Incorprar la categoria y el nombre. Cada elemento debe ir en una sola columna
df['atc1_code'] = df['atc5_code'].apply(lambda x: x[0])
df["atc1_name"] = df["atc1_code"].map(dict(zip(category_codes, category_names)))
df["atc2_code"] = df["atc5_code"].apply(lambda x: x[:3])
df["atc2_name"] = df["atc2_code"].map(dict(zip(category_codes, category_names)))
df["atc3_code"] = df["atc5_code"].apply(lambda x: x[:4])
df["atc3_name"] = df["atc3_code"].map(dict(zip(category_codes, category_names)))
df["atc4_code"] = df["atc5_code"].apply(lambda x: x[:5])
df["atc4_name"] = df["atc4_code"].map(dict(zip(category_codes, category_names)))

#Ordenar las columnas
df = df[['atc1_code', 'atc1_name', 'atc2_code', 'atc2_name', 'atc3_code', 'atc3_name', 'atc4_code', 'atc4_name', 'atc5_code', 'drug_name', 'ddd', 'unit', 'administration', 'note']]
#Minúsculas
df = df.apply(lambda x: x.str.lower() if x.dtype == "object" else x)
df

,atc1_code,atc1_name,atc2_code,atc2_name,atc3_code,atc3_name,atc4_code,atc4_name,atc5_code,drug_name,ddd,unit,administration,note
